Download requirements

In [2]:
!git clone --recurse-submodules https://github.com/batu/RLAgency.git

fatal: destination path 'RLAgency' already exists and is not an empty directory.


In [3]:
!pip install -e RLAgency/stable-baselines3/
!pip install mlagents
!pip install gym_unity
!pip install wandb

Obtaining file:///content/RLAgency/stable-baselines3
  Found existing installation: stable-baselines3 1.1.0a1
    Can't uninstall 'stable-baselines3'. No files were found to uninstall.
  Running setup.py develop for stable-baselines3


In [4]:
import os
from google.colab import drive
drive.mount('gdrive')

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


In [5]:
ENV_PATH = "/content/gdrive/MyDrive/RLNav/Builds/BasicSingle20/Env.x86_64"

In [6]:
!chmod -R 755 {ENV_PATH}

In [7]:
os.path.isfile(ENV_PATH)

True

# Reinforcement Learning for Navigation

In [8]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy


from pathlib import Path

from mlagents_envs.environment import UnityEnvironment
from gym_unity.envs import UnityToGymWrapper

In [10]:
unity_env = UnityEnvironment(str(ENV_PATH), base_port=6024)
env = UnityToGymWrapper(unity_env)
env.close()

2021-04-09 22:50:41 INFO [environment.py:113] Connected to Unity environment with package version 1.8.1-preview and communication version 1.4.0
2021-04-09 22:50:41 INFO [environment.py:282] Connected new brain:
Basic?team=0
2021-04-09 22:50:41 WARNING [__init__.py:92] The environment contains multiple observations. You must define allow_multiple_obs=True to receive them all. Otherwise, only the first visual observation (or vector observation ifthere are no visual observations) will be provided in the observation.
/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
2021-04-09 22:50:46 INFO [environment.py:429] Environment shut down with return code 0.


In [11]:
log_dir = "tmp/"
os.makedirs(log_dir, exist_ok=True)

def make_env(rank, seed=0):
    def _init():
        unity_env = UnityEnvironment(str(ENV_PATH), base_port=6142 + rank)
        env = UnityToGymWrapper(unity_env)
        env.seed(seed + rank)
        env = Monitor(env, log_dir)
        return env
    set_random_seed(seed)
    return _init

num_cpu = 4  # Number of processes to use
env = DummyVecEnv([make_env(i) for i in range(num_cpu)])

2021-04-09 22:50:51 INFO [environment.py:113] Connected to Unity environment with package version 1.8.1-preview and communication version 1.4.0
2021-04-09 22:50:51 INFO [environment.py:282] Connected new brain:
Basic?team=0
2021-04-09 22:50:51 WARNING [__init__.py:92] The environment contains multiple observations. You must define allow_multiple_obs=True to receive them all. Otherwise, only the first visual observation (or vector observation ifthere are no visual observations) will be provided in the observation.
/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
2021-04-09 22:50:51 WARNING [__init__.py:283] Could not seed environment Basic?team=0
2021-04-09 22:50:55 INFO [environment.py:113] Connected to Unity environment with package version 1.8.1-preview and communication version 1.4.0
2021-04-09 22:50:55 INFO [environment.py:282] Connected ne

In [12]:
model = PPO('MlpPolicy', env, verbose=2)

Using cuda device


In [ ]:
model.learn(total_timesteps=50000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 241      |
|    ep_rew_mean     | -0.189   |
| time/              |          |
|    fps             | 300      |
|    iterations      | 1        |
|    time_elapsed    | 54       |
|    total_timesteps | 16384    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 247         |
|    ep_rew_mean          | -0.27       |
| time/                   |             |
|    fps                  | 224         |
|    iterations           | 2           |
|    time_elapsed         | 145         |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.004214269 |
|    clip_fraction        | 0.0536      |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.23       |
|    explained_variance   | 0.173       |
|    learning_rate        | 0.

In [ ]:
evaluate_policy(model, model.get_env(), n_eval_episodes=3)

In [87]:
obs = env.reset()[0:1]
rew = 0
for _ in range(1000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.envs[0].step(action)
    rew += rewards
    if(dones):
      print("Episode complete!", rew)

Episode complete! -0.1328778862953186
Episode complete! -0.1650303453207016
Episode complete! -0.3981654495000839
